# Average the Multimeasurements for Reflection
## Abstract
- The Measuremnts for reflection where performed 3 at a time. Its necesarry to cut them up.
- Cutting is performed on the transformed Impulses

## Doing
- Load Impulse Group
- Cut it Up
- Upsample
- Average
- Write to Imp File

## Files
From: TARGET_DIR/IMP_REC{Nr}.wav --> To: ./{Measurement Pos}.wav

In [1]:
# 1. Import, Base Settings (System) and Base Settings (User)
from Z211014_reflection_definitions import *

%matplotlib notebook
plt.rcParams['axes.grid'] = True    # Set grid in plots on per default
VERBOSE = False
F_up = 500e3


############################################################
########## User Interaction Starts Here       ##############
########## Please specify where the Files are ##############
############################################################

TARGET_DIR = 'C:/Users/gmeinwieserch/Desktop/04  St-Martha Kirche/211007_Martha/Impulse/'
NAME = 'IMP_REC0%s.wav'

NR = {'Wand_0_0'  : ['27'], # LR =   0, H =   0, NeutrH = 1.6
      'Wand_+_0'  : ['28'], # LR = +.4, H =   0, NeutrH = 1.6
      'Wand_-_0'  : ['29'], # LR = -.4, H =   0, NeutrH = 1.6
      'Wand_0_+'  : ['30'], # LR =   0, H = +.4, NeutrH = 1.6
      'Wand_0_-'  : ['31'], # LR =   0, H = -.4, NeutrH = 1.6
      'Boden_1.5' : ['32'], # A  = 1.5, H = .93
      'Boden_.2'  : ['33'], # A  =  .2, H = .93
      'Direct_.8' : ['35']} # A  =  .8, H = 2

################################################################
######### Please specify the exitation pars        #############
################################################################
par_sweep = [63, 10, 5e3]   # parameter of sweep [fstart, T, fend]


In [2]:
for position in NR.keys():
    #position = 'Wand_0_0'#for position in NR.keys():
    all_imp = Signal(path = TARGET_DIR, name = NAME % (NR[position][0]))

    all_cut = []
    all_cut.append(all_imp.cut_signal(3,10))
    all_cut.append(all_imp.cut_signal(17,24))
    all_cut.append(all_imp.cut_signal(33,40))
    
    cut_up = [imp.resample(F_up) for imp in all_cut]
    rotate_sig_lst(cut_up)

    end_cut = int(3.2/cut_up[0].dt)   # min([el.n_tot for el in cut_up])
    imp_avg = Signal(signal_lst_imp = [sig.cut_signal(t_start=0, t_end=.1, force_n = end_cut) for sig in cut_up])
    imp_avg_down = imp_avg.resample(48e3)
    imp_avg_down.write_wav(name = 'IMP_' + position + '.wav',F_samp= 48e3)